**Configuração e Inicialização**

#  PREPARANDO O AMBIENTE

In [21]:
!pip install pyspark # Instala a biblioteca PySpark no ambiente de execução

from pyspark.sql import SparkSession # Importa a classe SparkSession, o ponto de entrada para o Spark
from pyspark.sql.functions import col, month, lit, when # col (referência a coluna), month (extrai o mês de uma data), lit (cria uma coluna com valor literal) e when (lógica condicional)
from pyspark.ml.feature import StringIndexer, VectorAssembler, Normalizer, PCA # StringIndexer (codificação categórica), VectorAssembler (agrupamento de features), Normalizer (normalização de vetor) e PCA (redução de dimensionalidade)
from pyspark.ml.regression import LinearRegression # Importa o modelo de Regressão Linear
from pyspark.ml.evaluation import RegressionEvaluator # Importa a classe para calcular métricas de regressão, como RMSE

In [22]:
# cria a SparkSession
# Cria ou obtém a SparkSession e define o nome da aplicação
spark = SparkSession.builder \
.appName("preparacao_ML") \
.getOrCreate()

print("SparkSession iniciada e biblioteca de ML importadas!") # Confirma o início da sessão
print(spark) # Exibe a instância da SparkSession

SparkSession iniciada e biblioteca de ML importadas!


**Leitura e Limpeza Inicial de Dados**

#  LEITURA DO DATAFRAME

In [23]:
df_video = spark.read.parquet("/content/videos-comments-tratados.snappy.parquet") # Carrega o DataFrame a partir de um arquivo Parquet pré-processado

print("DataFrame 'df_video' lido. Schema inicial:") # Confirma a leitura
df_video.printSchema() # Exibe a estrutura inicial (colunas e tipos) do DataFrame
print(f"total de registros: {df_video.count()}") # Mostra a contagem total de registros antes da limpeza final

DataFrame 'df_video' lido. Schema inicial:
root
 |-- Video ID: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Published At: date (nullable = true)
 |-- Keyword: string (nullable = true)
 |-- Likes: integer (nullable = true)
 |-- Comments: integer (nullable = true)
 |-- Views: integer (nullable = true)
 |-- Interaction: integer (nullable = true)
 |-- Year: string (nullable = true)
 |-- Comment: string (nullable = true)
 |-- Sentiment: integer (nullable = true)
 |-- Likes Comment: integer (nullable = true)

total de registros: 18409


In [24]:
# Remove nulos nas colunas que serão usadas como features (atributos)
df_video.na.drop(subset=['Year','Keyword','Likes','Views','Comments']) # Remove as linhas inteiras que contêm valores nulos nas colunas essenciais para o modelo (Year, Keyword, Likes, Views, Comments)
print(f"total de registros após limpeza final (drop nulos nas colunas chave): {df_video.count()}") # Mostra a contagem de registros após a remoção dos nulos nas colunas chave

total de registros após limpeza final (drop nulos nas colunas chave): 18409


**Feature Engineering e Codificação**

#  FEATURE ENGINEERING - COLUNA 'MONTH'

In [25]:
df_video = df_video.withColumn('Month', month(col("Published At"))) # Adiciona a coluna Month extraindo o mês da coluna de data de publicação

print("coluna 'month' adicionada com sucesso. Visualizando a data e o mês:") # Confirma a criação da coluna
df_video.select("published At", "Month").show(5) # Exibe a data e o mês extraído para verificação

coluna 'month' adicionada com sucesso. Visualizando a data e o mês:
+------------+-----+
|published At|Month|
+------------+-----+
|  2022-08-23|    8|
|  2022-08-23|    8|
|  2022-08-23|    8|
|  2022-08-23|    8|
|  2022-08-23|    8|
+------------+-----+
only showing top 5 rows



#  FEATURE ENGINEERING - COLUNA 'KEYWORD INDEX'

In [26]:
#Define o transformador para converter a coluna categórica Keyword em índices numéricos (codificação Label Encoding), pulando valores inválidos
indexer = StringIndexer(
    inputCol="Keyword",
    outputCol="Keyword Index",
    handleInvalid="skip" # Ignora registros com Keyword nulo (se houver algum)
)


df_video = indexer.fit(df_video).transform(df_video) # O fit calcula os índices e o transform aplica essa codificação, criando a coluna Keyword Index

print("coluna 'keyword Index' criada a partir de 'Keyword'.") # Confirma a criação da coluna
df_video.select("Keyword", "Keyword Index").show(5, truncate=False) # Exibe o mapeamento entre a palavra-chave original e seu índice numérico

coluna 'keyword Index' criada a partir de 'Keyword'.
+-------+-------------+
|Keyword|Keyword Index|
+-------+-------------+
|tech   |17.0         |
|tech   |17.0         |
|tech   |17.0         |
|tech   |17.0         |
|tech   |17.0         |
+-------+-------------+
only showing top 5 rows



**Preparação do Vetor de Features**

#  VECTORASSEMBLER - CRIAÇÃO DO VETOR 'FEATURES'

In [27]:
df_video = df_video.withColumn("Year_int", col("Year").cast("integer")) # Cria uma coluna auxiliar, Year_int, forçando o tipo de Year para Inteiro, garantindo que o VectorAssembler possa usá-la

features_col = ["Likes","Views","Year_int","Month","Keyword Index"] # Lista todas as colunas que farão parte do vetor de features

# Cria o VectorAssembler para juntar as colunas
# Define o transformador que irá combinar as colunas listadas em um único vetor chamado Features
assembler = VectorAssembler(
    inputCols= features_col,
    outputCol="Features"
)

df_video = assembler.transform(df_video) # Aplica o assembler ao DataFram

print(f"Colunas agrupadas: {features_col}") # Confirma as colunas usadas
print("coluna 'features' criada. Visualizando o vetor:") # Confirma a criação da coluna
df_video.select("Features").show(5, truncate=False) # Exibe o conteúdo vetorial da nova coluna Features

Colunas agrupadas: ['Likes', 'Views', 'Year_int', 'Month', 'Keyword Index']
coluna 'features' criada. Visualizando o vetor:
+---------------------------------+
|Features                         |
+---------------------------------+
|[3407.0,135612.0,2022.0,8.0,17.0]|
|[3407.0,135612.0,2022.0,8.0,17.0]|
|[3407.0,135612.0,2022.0,8.0,17.0]|
|[3407.0,135612.0,2022.0,8.0,17.0]|
|[3407.0,135612.0,2022.0,8.0,17.0]|
+---------------------------------+
only showing top 5 rows



**Normalização e Redução de Dimensionalidade**

#  NORMALIZAÇÃO - CRIAÇÃO DO VETOR 'FEATURES NORMAL'

In [28]:
# Define o normalizador que aplicará a norma L2 (p=2.0) no vetor Features, resultando em Features Normal (norma unitária)
normalizer = Normalizer(
    inputCol="Features",
    outputCol="Features Normal",
    p=2.0
)

df_video = normalizer.transform(df_video) # Aplica a normalização ao DataFrame

print("coluna 'features normal' criada (vetor normalizado).") # Confirma a criação da coluna
df_video.select("Features", "Features Normal").show(5, truncate=False) # Exibe o vetor original e o vetor normalizado.

coluna 'features normal' criada (vetor normalizado).
+---------------------------------+--------------------------------------------------------------------------------------------------------+
|Features                         |Features Normal                                                                                         |
+---------------------------------+--------------------------------------------------------------------------------------------------------+
|[3407.0,135612.0,2022.0,8.0,17.0]|[0.02511243093431334,0.9995735203592899,0.014903826049070024,5.896667081728991E-5,1.2530417548674107E-4]|
|[3407.0,135612.0,2022.0,8.0,17.0]|[0.02511243093431334,0.9995735203592899,0.014903826049070024,5.896667081728991E-5,1.2530417548674107E-4]|
|[3407.0,135612.0,2022.0,8.0,17.0]|[0.02511243093431334,0.9995735203592899,0.014903826049070024,5.896667081728991E-5,1.2530417548674107E-4]|
|[3407.0,135612.0,2022.0,8.0,17.0]|[0.02511243093431334,0.9995735203592899,0.014903826049070024,5.896

#  REDUÇÃO DE DIMENSIONALIDADE (PCA)

In [29]:
# Define o PCA para reduzir o vetor normalizado para 1 componente principal (k=1)
pca = PCA(
    k=1, # Quero apenas 1 componente principal
    inputCol="Features Normal",
    outputCol="Features PCA"
)

modelo_pca = pca.fit(df_video) # O fit calcula o componente principal dos dados
df_video = modelo_pca.transform(df_video) # Transforma os Dados: Aplica o PCA, gerando o vetor Features PCA

print("coluna 'features PCA' criada com 1 componente principal.") # Confirma a criação da coluna
print(f"Variância explicada (importância do componente): {modelo_pca.explainedVariance}") # Imprime a variância explicada por esse único componente, indicando o quanto ele retém da informação original
df_video.select("Features Normal", "Features PCA").show(5, truncate=False) # Exibe o vetor normalizado e o vetor reduzido (PCA)

coluna 'features PCA' criada com 1 componente principal.
Variância explicada (importância do componente): [0.9010786618805576]
+--------------------------------------------------------------------------------------------------------+---------------------+
|Features Normal                                                                                         |Features PCA         |
+--------------------------------------------------------------------------------------------------------+---------------------+
|[0.02511243093431334,0.9995735203592899,0.014903826049070024,5.896667081728991E-5,1.2530417548674107E-4]|[0.39522435928489963]|
|[0.02511243093431334,0.9995735203592899,0.014903826049070024,5.896667081728991E-5,1.2530417548674107E-4]|[0.39522435928489963]|
|[0.02511243093431334,0.9995735203592899,0.014903826049070024,5.896667081728991E-5,1.2530417548674107E-4]|[0.39522435928489963]|
|[0.02511243093431334,0.9995735203592899,0.014903826049070024,5.896667081728991E-5,1.25304175486741

# REGRESSÃO LINEAR E AVALIAÇÃO

In [30]:
# 1 Separar os dados em treino e teste (80/20)
train_data, test_data = df_video.randomSplit([0.8,0.2], seed=42) # Separa os dados em conjuntos de treinamento (80%) e teste (20%)
print(f"dados de treinamento: {train_data.count()} registros") # Exibe a contagem de registros de treinamento
print(f"dados de teste: {test_data.count()} registros") # Exibe a contagem de registros de teste.

dados de treinamento: 14789 registros
dados de teste: 3620 registros


In [31]:
# 2 Criar e Treinar o Modelo
# Define o modelo. As features de entrada são o vetor Features Normal e o rótulo a ser previsto é o número de Comments
lr = LinearRegression(
    featuresCol="Features Normal",
    labelCol="Comments"
)

print("\nTreinando o modelo de regressão linear...") # Mensagem de status
lr_model = lr.fit(train_data) # Treina o modelo usando o conjunto de dados de treinamento


Treinando o modelo de regressão linear...


In [33]:
# 3 Fazer Previsões no conjunto de teste
predictions = lr_model.transform(test_data) # gera as previsões no conjunto de teste

In [32]:
# 4 Avaliar o Modelo
# Usei o RMSE (Root Mean Square Error) para avaliar regressão
evaluator = RegressionEvaluator(
    labelCol="Comments", #
    predictionCol="prediction", #
    metricName="rmse" # Raiz do Erro Quadrático Médio
)

rmse = evaluator.evaluate(predictions) # Calcula o valor do RMSE no conjunto de teste.
print(f"\nRMSE (root mean square error) no conjunto de teste: {rmse:.2f}") # Imprime o resultado do RMSE


RMSE (root mean square error) no conjunto de teste: 43345.23


#SALVAMENTO DO DATAFRAME FINAL

In [34]:
df_video.write.mode("overwrite").parquet("videos-preparados-parquet") # Exporta o DataFrame (agora com todas as features de ML) para o formato Parquet

print("dataframe 'df_video' salvo com sucesso como 'videos-preparados-parquet'.") # Confirma o salvamento

print("\n\n--- PREPARAÇÃO DE DADOS CONCLUÍDA! ---") # Mensagem de conclusão

dataframe 'df_video' salvo com sucesso como 'videos-preparados-parquet'.


--- PREPARAÇÃO DE DADOS CONCLUÍDA! ---
